In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from pathlib import Path

## Preprocessing

In [ ]:
dir_path = "/workspaces/gorillatracker/datasets/splits/-workspaces-gorillatracker-data-ground_truth-cxl_all-face_images-kfold-actual_openset-seed-42-trainval-80-test-20-k-5"

In [ ]:
dirs = os.listdir(dir_path)
print(dirs)

In [ ]:
#create a dataframe for each directory
df: list[pd.DataFrame] = []

for i, d in enumerate(dirs):
    data = []
    solitary_data = 0
    # Extract data from filenames
    for filename in os.listdir(os.path.join(dir_path, d)):
        parts = filename.split('_')
        if len(parts) >= 4:
            id_, camid, date, id2 = parts[:4]
            solitary = False
            if(len(id_) != 4):  # if != 4, then the gorilla is a solitary one without a group
                # print(f"Solitary in {filename}")
                solitary_data += 1
                solitary = True
            Groupid = id_[:2] if not solitary else "SOLITARY"
            id_ = id_[2:] if not solitary else ''.join(filter(str.isdigit, id_))
            id2 = ''.join(filter(str.isdigit, id2)) # remove non-numeric characters
            date = dt.datetime.strptime(date, '%Y%m%d').date()
            data.append((Groupid, id_, camid, date, id2, f"{dir_path}/{d}/{filename}"))
        else:
            print(f"Invalid filename {filename}")
            
    # Create a DataFrame
    df.append(pd.DataFrame(data, columns=['GROUP', 'ID', 'CAM', 'DATE', 'ID2', 'FILENAME']))
    print(f"Directory {d} had {solitary_data} images of solitary gorillas with no group")

In [ ]:
print(df[2]["FILENAME"][0])

## Plot the Number of Videos per Individual

In [ ]:
for i, d in enumerate(dirs):
    print(f"{d}:")
    print(f"\t Images: {df[i].shape[0]}")
    print(f"\t Individuals: {df[i].groupby(['GROUP', 'ID']).ngroups}")
    print(f"\t Groups: {df[i]['GROUP'].nunique()}")
    print(f"\t Cameras: {df[i]['CAM'].nunique()}")
    print(f"\t Days: {df[i]['DATE'].nunique()}")
    print(f"\t DateRange: {df[i]['DATE'].min()} - {df[i]['DATE'].max()}")
    print(f"\t Video_clips: {df[i].groupby(['CAM', 'DATE', 'ID2']).ngroups}")
    print(f"\t Videos: {df[i].groupby(['CAM', 'DATE']).ngroups}")
    
df_combined = pd.concat(df)
print(f"Combined:")
print(f"\t Images: {len(df_combined)}")
print(f"\t Individuals: {df_combined.groupby(['GROUP', 'ID']).ngroups}")
print(f"\t Groups: {df_combined['GROUP'].nunique()}")
print(f"\t Cameras: {df_combined['CAM'].nunique()}")
print(f"\t Days: {df_combined['DATE'].nunique()}")
print(f"\t DateRange: {df_combined['DATE'].min()} - {df_combined['DATE'].max()}")
print(f"\t Video_clips: {df_combined.groupby(['CAM', 'DATE', 'ID2']).ngroups}")
print(f"\t Videos: {df_combined.groupby(['CAM', 'DATE']).ngroups}")

unique_videos = df_combined.drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE', 'ID2'])
videos_per_id = unique_videos.groupby(['GROUP', 'ID']).size()
videos_count_distribution = videos_per_id.value_counts().sort_index()
plt.figure(figsize=(5, 2))
plt.bar(videos_count_distribution.index, videos_count_distribution.values)
plt.xlabel('Num Videos/ID')
plt.ylabel('Individuals')
plt.title(f'Complete DS: Number of Videos per ID')
plt.xticks(videos_count_distribution.index)
plt.show()

In [ ]:
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE'])
    videos_per_id = unique_videos.groupby(['GROUP', 'ID']).size()
    videos_count_distribution = videos_per_id.value_counts().sort_index()
    plt.figure(figsize=(5, 2))
    plt.bar(videos_count_distribution.index, videos_count_distribution.values)
    plt.xlabel('Num Videos/ID')
    plt.ylabel('Individuals')
    plt.title(f'{d}: Number of Videos per ID')
    plt.xticks(videos_count_distribution.index)
    plt.show()


In [ ]:
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE', 'ID2'])
    videos_per_id = unique_videos.groupby(['GROUP', 'ID']).size()
    videos_count_distribution = videos_per_id.value_counts().sort_index()
    plt.figure(figsize=(5, 2))
    plt.bar(videos_count_distribution.index, videos_count_distribution.values)
    plt.xlabel('Num VideoClips/ID')
    plt.ylabel('Individuals')
    plt.title(f'{d}: Number of Video Clips per ID')
    plt.xticks(videos_count_distribution.index)
    plt.show()

## Number of Images of Individuals with at least 2 Videos

In [ ]:
df_combined = pd.concat(df)
unique_videos = df_combined.drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE'])

videos_per_id = unique_videos[['GROUP', 'ID']].copy()
videos_per_id = videos_per_id.groupby(['GROUP', 'ID']).size().reset_index()
images_per_video = df_combined[["GROUP", "ID"]].copy()
images_per_video = images_per_video.groupby(['GROUP', 'ID']).size()
images_per_video.name = "IMAGES"

video_image_distribution = videos_per_id.join(images_per_video, on=["GROUP", "ID"], how="inner")
video_image_distribution["VIDEOS"] = video_image_distribution[0]
video_image_distribution = video_image_distribution.drop(columns=[0])

# plot num_images by number of videos
plt.figure(figsize=(5, 5))
plt.scatter(video_image_distribution["VIDEOS"], video_image_distribution["IMAGES"])
plt.xlabel('Num Videos')
plt.ylabel('Num Images')
plt.title(f'Complete DS: Num Images by Num Videos')
plt.show()

# number of images per video count
plt.figure(figsize=(5, 5))
plt.bar(video_image_distribution["VIDEOS"].value_counts().index, video_image_distribution.groupby(video_image_distribution["VIDEOS"])["IMAGES"].sum().sort_index().values)
plt.xlabel('Num Videos')
plt.ylabel('Num Images')
plt.title(f'Complete DS: Num Images by Num Videos')
plt.show()



for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE'])
    videos_per_id = unique_videos[['GROUP', 'ID']].copy()
    videos_per_id = videos_per_id.groupby(['GROUP', 'ID']).size().reset_index()
    images_per_video = df[i][["GROUP", "ID"]].copy()
    images_per_video = images_per_video.groupby(['GROUP', 'ID']).size()
    images_per_video.name = "IMAGES"
    video_image_distribution = videos_per_id.join(images_per_video, on=["GROUP", "ID"], how="inner")
    video_image_distribution["VIDEOS"] = video_image_distribution[0]
    video_image_distribution = video_image_distribution.drop(columns=[0])
    # plot num_images by number of videos
    plt.figure(figsize=(5, 5))
    plt.scatter(video_image_distribution["VIDEOS"], video_image_distribution["IMAGES"])
    plt.xlabel('Num Videos')
    plt.ylabel('Num Images')
    plt.title(f'{d}: Num Images by Num Videos')
    plt.show()
    
    plt.figure(figsize=(5, 5))
    plt.bar(video_image_distribution["VIDEOS"].value_counts().index, video_image_distribution.groupby(video_image_distribution["VIDEOS"])["IMAGES"].sum().sort_index().values)
    plt.xlabel('Num Videos')
    plt.ylabel('Num Images')
    plt.title(f'{d}: Num Images by Num Videos')
    plt.show()



## Number of Images per Video and per Individual # TODO

In [ ]:
from collections import defaultdict

combined_ids = df_combined["GROUP"] + df_combined["ID"]
all_individuals = set(combined_ids.unique())
individual_seen_in = defaultdict(set)
print(dirs)
for i, d in enumerate(dirs):
    print(f"Processing {d}")
    individuals = set((df[i]["GROUP"] + df[i]["ID"]).unique())
    for ind in individuals:
        individual_seen_in[ind].add(d)
        
for ind, aDir in individual_seen_in.items():
    if len(aDir) > 1:
        print(f"{ind} seen in {aDir}")
print(individual_seen_in)

In [ ]:
import numpy as np

df_combined = pd.concat(df)
unique_videos = df_combined.drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE'])
videos_per_id = unique_videos[['GROUP', 'ID', 'CAM', 'DATE']].copy().groupby(['GROUP', 'ID']).size().reset_index()

images_per_video = df_combined[["GROUP", "ID", 'CAM', 'DATE']].copy()
images_per_video = images_per_video.groupby(['GROUP', 'ID', 'CAM', 'DATE']).size()
images_per_video.name = "IMAGES"

images_per_video = images_per_video.reset_index().join(videos_per_id.set_index(['GROUP', 'ID']), on=['GROUP', 'ID'], how='inner')
images_per_video["VIDEOS"] = images_per_video[0]
images_per_video = images_per_video.drop(columns=[0])
images_per_video = images_per_video.reset_index()
images_per_video["COMPLETE_ID"] = images_per_video["GROUP"] + images_per_video["ID"]
images_per_video = images_per_video.drop(columns=["GROUP", "ID"])

plt.figure(figsize=(5, 5))
plt.scatter(images_per_video["VIDEOS"], images_per_video["IMAGES"], c=images_per_video["COMPLETE_ID"].astype('category').cat.codes, cmap='tab20')
plt.colorbar()
plt.xlabel('Num Videos')
plt.ylabel('Num Images')
plt.show()

print(dirs)
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE'])
    videos_per_id = unique_videos[['GROUP', 'ID', 'CAM', 'DATE']].copy().groupby(['GROUP', 'ID']).size().reset_index()
    images_per_video = df[i][["GROUP", "ID", 'CAM', 'DATE']].copy()
    images_per_video = images_per_video.groupby(['GROUP', 'ID', 'CAM', 'DATE']).size()
    images_per_video.name = "IMAGES"
    images_per_video = images_per_video.reset_index().join(videos_per_id.set_index(['GROUP', 'ID']), on=['GROUP', 'ID'], how='inner')
    images_per_video["VIDEOS"] = images_per_video[0]
    images_per_video = images_per_video.drop(columns=[0])
    images_per_video = images_per_video.reset_index()
    images_per_video["COMPLETE_ID"] = images_per_video["GROUP"] + images_per_video["ID"]
    images_per_video = images_per_video.drop(columns=["GROUP", "ID"])
    print(f"{d}:")
    print(images_per_video)

    plt.figure(figsize=(5, 10))
    jitter = np.random.normal(0, 0.1, images_per_video.shape[0])
    plt.scatter(images_per_video["VIDEOS"] + jitter, images_per_video["IMAGES"], c=images_per_video["COMPLETE_ID"].astype('category').cat.codes, cmap='tab20', alpha=0.8, s=100, edgecolors='black')
    # set a legend to map id to color
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=plt.cm.tab20.colors[i % len(plt.cm.tab20.colors)], label=id_) for i, id_ in enumerate(images_per_video["COMPLETE_ID"].unique())]
    # place the legend outside the plot
    plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # TODO
    plt.xlabel('Num Videos')
    plt.ylabel('Num Images')
    plt.title(f'{d}: Num Images by Num Videos')
    plt.show()

## Number of Cameras per Individual

In [ ]:
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM'])
    videos_per_id = unique_videos.groupby(['GROUP', 'ID']).size()
    videos_count_distribution = videos_per_id.value_counts().sort_index()
    plt.figure(figsize=(5, 2))
    plt.bar(videos_count_distribution.index, videos_count_distribution.values)
    plt.xlabel('Num Cam/ID')
    plt.ylabel('Individuals')
    plt.title(f'{d}: Number of Cameras per ID')
    plt.xticks(videos_count_distribution.index)
    plt.show()

## Images per Camera

In [ ]:
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE', 'ID2'])
    images_per_camera = unique_videos.groupby('CAM').size()
    plt.figure(figsize=(10, 5))
    plt.bar(images_per_camera.index, images_per_camera.values)
    plt.xlabel('Camera ID')
    plt.ylabel('Images')
    plt.title(f'{d}: Images/Camera')
    plt.show()

## Individuals per Camera

In [ ]:
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE', 'ID2'])
    individuals_per_camera = unique_videos.groupby(['CAM', 'GROUP', 'ID']).size().reset_index().groupby('CAM').size()
    plt.figure(figsize=(10, 5))
    plt.bar(individuals_per_camera.index, individuals_per_camera.values)
    plt.xlabel('Camera ID')
    plt.ylabel('Individuals')
    plt.title(f'{d}: Individuals/Camera')
    plt.show()

In [ ]:
# combined
df_combined = pd.concat(df)
unique_videos = df_combined.drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE', 'ID2'])
images_per_camera = unique_videos.groupby('CAM').size()
plt.figure(figsize=(10, 5))
plt.bar(images_per_camera.index, images_per_camera.values)
plt.xlabel('Camera ID')
plt.ylabel('Images')
plt.title('Combined: Images/Camera')
plt.xticks(rotation=90)
plt.show()

# make this plot a histogram with bins of 10 images per camera and show the number of cameras in each bin
# center the bins
plt.figure(figsize=(10, 5))
plt.hist(images_per_camera, bins=range(0, 80, 5), align='left', rwidth=0.9)
plt.xticks(ticks=range(0, 80, 5), labels=[f'{i} - {i+5}' for i in range(0, 80, 5)], rotation=90)
plt.xlabel('Images/Camera')
plt.ylabel('Cameras')
plt.title('Combined: Images/Camera')
plt.show()

## Individuals per Group

In [ ]:
for i, d in enumerate(dirs):
    unique_videos = df[i].drop_duplicates(subset=['GROUP', 'ID', 'CAM', 'DATE', 'ID2'])
    unique_individuals_per_group = unique_videos.groupby(['GROUP', 'ID']).size().reset_index().groupby('GROUP').size()
    plt.figure(figsize=(10, 5))
    plt.bar(unique_individuals_per_group.index, unique_individuals_per_group.values)
    plt.xlabel('Group ID')
    plt.ylabel('Individuals')
    plt.title(f'{d}: Individuals/Group')
    plt.xticks(rotation=90)
    plt.show()

## Images per Category

In [ ]:
def categorize_id(id_):
    id_num = int(id_)
    if id_num == 0:
        return 'Silverback'
    elif 1 <= id_num <= 19:
        return 'Adult female'
    elif 20 <= id_num <= 39:
        return 'Blackback'
    elif 40 <= id_num <= 59:
        return 'Adolescent & Juvenil'
    elif 60 <= id_num <= 79:
        return 'Infant'
    else:
        return 'Unknown'

for i, d in enumerate(dirs):
    unique_individuals = df[i].drop_duplicates(subset=['GROUP', 'ID']).copy()
    # filter out non numeric IDs
    unique_individuals = unique_individuals[unique_individuals['ID'].str.isnumeric()]
    
    # Categorize each ID and add to the DataFrame using assign()
    unique_individuals = unique_individuals.assign(Type=unique_individuals['ID'].apply(categorize_id))
    
    # Calculate the number of each type
    type_distribution = unique_individuals['Type'].value_counts().sort_index()
    
    
    # Plot the distribution of types
    plt.figure(figsize=(10, 5))
    plt.bar(type_distribution.index, type_distribution.values)
    plt.xlabel('Type')
    plt.ylabel('Number of Individuals')
    plt.title(f'{d}: Types of Individuals')
    plt.show()

# Resolution of Images

In [ ]:
# plot the resolution of all combined images 

from PIL import Image

img_sizes = []
for row in df_combined.itertuples():
    img = Image.open(os.path.join(dir_path, row[6]))
    img_sizes.append(img.size)
        
len(img_sizes)

In [ ]:
# make a matrix histogram of the image sizes (width x height)
from matplotlib.colors import Colormap, LogNorm
import numpy as np

plt.figure(figsize=(10, 5))
amounts_x, amounts_y = [x[0] for x in img_sizes], [x[1] for x in img_sizes]
norm = LogNorm(vmax=1000)
plt.hist2d(amounts_x, amounts_y, bins=range(0, 500, 50), cmap="Greens", norm=norm)
for i in range(0, 450, 50):
    for j in range(0, 450, 50):
        plt.text(i+25, j+25, f"{len([1 for x in img_sizes if i <= x[0] < i+50 and j <= x[1] < j+50])}", color='black', ha='center', va='center')
plt.xlabel('Width')
plt.ylabel('Height')
plt.title('Image Sizes')
# plt.colorbar()
plt.grid()
plt.show()

In [ ]:
# plot one low mid and high resolution image
very_low_res = [x for x in img_sizes if x[0] < 50 and x[1] < 50][1]
low_res = [x for x in img_sizes if 50 <= x[0] < 100 and 50 <= x[1] < 100][0]
mid_res = [x for x in img_sizes if 100 <= x[0] < 200 and 100 <= x[1] < 200][0]
high_res = [x for x in img_sizes if x[0] > 200 and x[1] > 200][21]

plt.figure(figsize=(20, 5))
plt.subplot(1, 4, 1)
plt.imshow(Image.open(os.path.join(dir_path, df_combined.iloc[img_sizes.index(very_low_res)]['FILENAME'])))
plt.title(f"Very Low Resolution: {very_low_res}")
plt.axis('off')
plt.subplot(1, 4, 2)
plt.imshow(Image.open(os.path.join(dir_path, df_combined.iloc[img_sizes.index(low_res)]['FILENAME'])))
plt.title(f"Low Resolution: {low_res}")
plt.axis('off')
plt.subplot(1, 4, 3)
plt.imshow(Image.open(os.path.join(dir_path, df_combined.iloc[img_sizes.index(mid_res)]['FILENAME'])))
plt.title(f"Mid Resolution: {mid_res}")
plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(Image.open(os.path.join(dir_path, df_combined.iloc[img_sizes.index(high_res)]['FILENAME'])))
plt.title(f"High Resolution: {high_res}")
plt.axis('off')
plt.show() 